In [1]:
import os
import json
import torch
import urllib
import random
import torch.nn as nn
import numpy as np
import pandas as pd

from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [2]:
## data download
url = "https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json"
urllib.request.urlretrieve(url, 'sarcasm.json')

with open('./sarcasm.json') as f:
    datas = json.load(f)

In [3]:
df = pd.DataFrame(datas)
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [4]:
## Tokenize example
## 문장을 token화한다.
sample = "Hello, my name is minjun kim."
tokenizer = get_tokenizer("basic_english")
tokenizer(sample)

['hello', ',', 'my', 'name', 'is', 'minjun', 'kim', '.']

In [5]:
def yield_tokens(sentences):
    for text in sentences:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(df['headline'].tolist()),
                                  specials=["<Unknown>"], ## 어휘에 없는 단어들을 "<Unknown>"로 대체
                                  min_freq=2,
                                  max_tokens=1000,)
vocab.set_default_index(vocab['<Unknown>']) ## 생성된 어휘에서 "<UNK>" 토큰을 기본 인덱스로 설정

str_to_idx = vocab.get_stoi()
idx_to_str = vocab.get_itos()

print(idx_to_str)
vocab(tokenizer(sample))

['<Unknown>', "'", 'to', 'of', 'the', 's', 'in', ',', 'for', 'a', 'on', '.', 'and', 'with', 'is', 'trump', 'new', 'man', 'from', 'at', 'you', 't', 'it', 'about', 'this', 'by', 'after', '?', 'be', 'that', 'how', 'out', 'he', 'as', 'up', 'not', 'what', 'can', 'are', 'your', 'his', 'who', 'just', 'has', 'will', 'more', 'all', 'one', 'into', 'report', 'i', 'why', 'have', 'area', 'woman', 'over', 'donald', 'u', 'says', 'day', 'obama', 'time', 'no', 'first', 'like', 'people', 'women', 'get', 'her', 'we', 'world', 'an', 'now', 'nation', 'house', 'life', 'off', 'clinton', 'they', 'make', 'still', 'than', 'was', 'my', 'white', 'back', 'down', 'if', 'when', 'family', 'could', 'she', 'their', 'do', 'before', 'americans', 'gop', 'most', 'way', '5', 'black', 'year', 'here', 'study', 'years', 'bill', 'should', 'would', 'him', 'president', 'best', 'so', 'america', 'police', 'only', 'watch', 'school', 'show', 'american', 'really', 'being', 'but', 'know', 'home', 'mom', 'things', 'death', 'during', 'go

[0, 7, 83, 347, 14, 0, 474, 11]

In [6]:
x_train, x_test, y_train, y_test = train_test_split(df["headline"],
                                                    df['is_sarcastic'],
                                                    stratify=df['is_sarcastic'],
                                                    test_size=0.2)

print(len(x_train[0]))
print(x_train[0])

print(len(x_train[1]))
print(x_train[1])

78
former versace store clerk sues over secret 'black code' for minority shoppers
84
the 'roseanne' revival catches up to our thorny political mood, for better and worse


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(device)

class CustomDataset(Dataset):
    def __init__(self, texts, labels, vocab, tokenizer):
        super().__init__()
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]

        return self.vocab(self.tokenizer(text)), label
    
def collate_batch(batch, max_sequence_length):
    label_list, text_list = [], []

    for text, label in batch:
        processed_text = torch.tensor(text[:max_sequence_length], dtype=torch.int64) ## 길이를 max_sequence_length를 넘지 못하게 만든다.
        text_list.append(processed_text)
        label_list.append(label)

    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list, batch_first=True, padding_value=0) ## padding을 통해 데이터의 길이를 일정하게 맞춰준다.

    return text_list.to(device), label_list.to(device)

cuda


In [8]:
train_dataset = CustomDataset(x_train, y_train, vocab=vocab, tokenizer=tokenizer)
valid_dataset = CustomDataset(x_test, y_test, vocab=vocab, tokenizer=tokenizer)

for data in train_dataset:
    print(data)
    break

([96, 568, 415, 0, 43, 45, 0, 664, 139, 229, 829], 0)


In [9]:
MAX_SEQUENCE_LEN = 100
BATCH_SIZE = 32

train_dataloader = DataLoader(train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              collate_fn=lambda x : collate_batch(x, MAX_SEQUENCE_LEN)) ## 최대 길이가 100

valid_dataloader = DataLoader(valid_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              collate_fn=lambda x : collate_batch(x, MAX_SEQUENCE_LEN))

for data in train_dataloader:
    tokens, labels = data[0], data[1]
    print(tokens.shape, labels.shape)
    print(tokens[0])
    break

torch.Size([32, 22]) torch.Size([32])
tensor([ 17, 926,  66,   6,   0, 615, 107, 265, 108, 631,   0,   0,   0,   6,
        827,   0,   0,   0,   0,   0,   0,   0], device='cuda:0')


In [10]:
NUM_VOCAB = len(vocab)
print(len(vocab))

x, y = next(iter(train_dataloader))
x = x.to(device)
y = y.to(device)

1000


In [11]:
EMBEDDING_DIM = 30
embedding = nn.Embedding(len(vocab), EMBEDDING_DIM).to(device) ## len(vocab)개의 단어들을 EMBEDDING_DIM 크기의 실수 벡터로 변환하는 임베딩을 생성.
print(embedding)

embedding_out = embedding(x)
print(embedding_out.shape)
print(embedding_out[0])

Embedding(1000, 30)
torch.Size([32, 20, 30])
tensor([[ 5.6780e-01, -1.8588e-01, -1.0390e+00, -1.5141e+00,  4.4408e-01,
         -1.3901e-01, -9.3228e-01, -1.7407e+00, -1.9198e+00, -1.3429e+00,
          4.8739e-01, -1.0344e+00,  1.9785e+00, -9.5842e-02, -5.3842e-01,
         -2.1118e-03,  3.5626e-01, -1.4527e+00, -1.8026e-01,  8.5381e-01,
          8.5727e-01, -7.3970e-01, -5.4882e-01,  8.9367e-01,  5.1219e-01,
          7.1578e-01,  6.1685e-01,  1.9030e-01,  1.4914e+00, -1.3061e+00],
        [ 5.6780e-01, -1.8588e-01, -1.0390e+00, -1.5141e+00,  4.4408e-01,
         -1.3901e-01, -9.3228e-01, -1.7407e+00, -1.9198e+00, -1.3429e+00,
          4.8739e-01, -1.0344e+00,  1.9785e+00, -9.5842e-02, -5.3842e-01,
         -2.1118e-03,  3.5626e-01, -1.4527e+00, -1.8026e-01,  8.5381e-01,
          8.5727e-01, -7.3970e-01, -5.4882e-01,  8.9367e-01,  5.1219e-01,
          7.1578e-01,  6.1685e-01,  1.9030e-01,  1.4914e+00, -1.3061e+00],
        [-4.5078e-01,  3.2779e-01, -1.8165e+00, -1.4904e+00, -1.3

In [12]:
HIDDEN_SIZE = 64
NUM_LAYERS = 1
BIDIRECTIONAL = 1
SEQ_LENGTH = x.size(1)

lstm = nn.LSTM(input_size=EMBEDDING_DIM, hidden_size=HIDDEN_SIZE, batch_first=True, device=device)
lstm

LSTM(30, 64, batch_first=True)

In [13]:
h_0 = torch.zeros(NUM_LAYERS * BIDIRECTIONAL, SEQ_LENGTH, HIDDEN_SIZE).to(device)
c_0 = torch.zeros(NUM_LAYERS * BIDIRECTIONAL, SEQ_LENGTH, HIDDEN_SIZE).to(device)

In [14]:
lstm_out, (hidden, cell) = lstm(embedding_out)
print(lstm_out.shape)
print(hidden.shape, cell.shape)

torch.Size([32, 20, 64])
torch.Size([1, 32, 64]) torch.Size([1, 32, 64])


In [15]:
def EmbeddingLSTM(x, vocab_size, embedding_dim, hidden_size, bidrectional, num_layers, device):
    print(f"Input Shape : {x.shape}")
    x = x.to(device)
    
    
    embedding = nn.Embedding(vocab_size, embedding_dim, device=device)
    embedding_out = embedding(x)
    print(f"Embedded Input Shape : {embedding_out.shape}")

    lstm = nn.LSTM(input_size=embedding_dim,
                   hidden_size=hidden_size,
                   num_layers=num_layers,
                   batch_first=True,
                   device=device)
    
    bidi = 2 if bidrectional else 1
    out, (h, c) = lstm(embedding_out)
    print(f"Output Shape : {out.shape}")
    print(f"Hidden Shape : {h.shape}")
    print(f"Cell State Shape : {c.shape}")

In [16]:
EmbeddingLSTM(x, vocab_size=len(vocab), embedding_dim=30, hidden_size=64, bidrectional=False, num_layers=2, device=device)

Input Shape : torch.Size([32, 20])
Embedded Input Shape : torch.Size([32, 20, 30])
Output Shape : torch.Size([32, 20, 64])
Hidden Shape : torch.Size([2, 32, 64])
Cell State Shape : torch.Size([2, 32, 64])


In [17]:
class TextClassificationModel(nn.Module):
    def __init__(self, num_classes, vocab_size, embedding_dim, hidden_size, num_layers, bidirectional=True, drop_prob=0.1):
        super(TextClassificationModel, self).__init__()
        self.num_classes = num_classes 
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = 2 if bidirectional else 1
        
        self.embedding = nn.Embedding(num_embeddings=vocab_size, 
                                      embedding_dim=embedding_dim)
        
        self.lstm = nn.LSTM(input_size=embedding_dim, 
                            hidden_size=hidden_size, 
                            num_layers=num_layers, 
                            batch_first=True,
                            bidirectional=bidirectional,
                           )
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.relu = nn.ReLU()
        
        self.fc = nn.Linear(hidden_size*self.bidirectional, hidden_size)
        self.output = nn.Linear(hidden_size, num_classes)
        
    def init_hidden_and_cell_state(self, batch_size, device):
        # LSTM 입력시 초기 Cell 에 대한 가중치 초기화를 진행합니다.
        # (num_layers*bidirectional, batch_size, hidden_size)
        self.hidden_and_cell = (
            torch.zeros(self.num_layers*self.bidirectional, batch_size, self.hidden_size).to(device),
            torch.zeros(self.num_layers*self.bidirectional, batch_size, self.hidden_size).to(device),
        )
        
    def forward(self, x):
        x = self.embedding(x)
        output, (h, c) = self.lstm(x, self.hidden_and_cell)
        # (batch_size, seq_length, hidden_size*bidirectional)
        # last sequence 의 (batch_size, hidden_size*bidirectional)
        h = output[:, -1, :]
        o = self.dropout(h)
        o = self.relu(self.fc(o))
        o = self.dropout(o)
        return self.output(o)

In [18]:
config = {
    'num_classes': 2, 
    'vocab_size': len(vocab),
    'embedding_dim': 16, 
    'hidden_size': 32, 
    'num_layers': 2, 
    'bidirectional': True,
}

model = TextClassificationModel(**config)
model.to(device)

TextClassificationModel(
  (embedding): Embedding(1000, 16)
  (lstm): LSTM(16, 32, num_layers=2, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (relu): ReLU()
  (fc): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=2, bias=True)
)

In [19]:
# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [20]:
def train(model, data_loader, loss_fn, optimizer, device):
    model.train()

    running_loss = 0
    corr = 0
    counts = 0
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)
    for idx, (txt, lbl) in enumerate(prograss_bar):
        txt = txt.to(device)
        lbl = lbl.to(device)
        
        optimizer.zero_grad()
        model.init_hidden_and_cell_state(len(txt), device)
        
        output = model(txt)
        
        loss = loss_fn(output, lbl)
        
        loss.backward()
        
        optimizer.step()
        
        output = output.argmax(dim=1)
        
        corr += (output == lbl).sum().item()
        counts += len(lbl)
        
        running_loss += loss.item()
        
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")
    
    acc = corr / len(data_loader.dataset)

    return running_loss / len(data_loader), acc

In [21]:
def evaluate(model, data_loader, loss_fn, device):
    model.eval()
    with torch.no_grad():
        corr = 0
        running_loss = 0
        
        for txt, lbl in data_loader:
            txt = txt.to(device)
            lbl = lbl.to(device)
            model.init_hidden_and_cell_state(len(txt), device)
    
            output = model(txt)
            loss = loss_fn(output, lbl)
            output = output.argmax(dim=1)

            corr += (output == lbl).sum().item()
            running_loss += loss.item()
        
        acc = corr / len(data_loader.dataset)
        return running_loss / len(data_loader), acc

In [23]:
num_epochs = 100
model_name = 'LSTM-Text-Classification'

min_loss = np.inf
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_dataloader, loss_fn, optimizer, device)
    val_loss, val_acc = evaluate(model, valid_dataloader, loss_fn, device)   
    
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss

        try:
            torch.save(model.state_dict(), f'/home/pervinco/Models/LSTM/{model_name}.pth')
        except:
            os.makedirs('/home/pervinco/Models/LSTM')
    
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

training loss: 0.51338, training accuracy: 0.74840: 100%|██████████| 668/668 [00:01<00:00, 469.95batch/s]


[INFO] val_loss has been improved from inf to 0.44937. Saving Model!
epoch 01, loss: 0.51338, acc: 0.74840, val_loss: 0.44937, val_accuracy: 0.78940


training loss: 0.44191, training accuracy: 0.79506: 100%|██████████| 668/668 [00:01<00:00, 459.89batch/s]


[INFO] val_loss has been improved from 0.44937 to 0.42500. Saving Model!
epoch 02, loss: 0.44191, acc: 0.79506, val_loss: 0.42500, val_accuracy: 0.80513


training loss: 0.40298, training accuracy: 0.81556: 100%|██████████| 668/668 [00:01<00:00, 470.75batch/s]


...

epoch 98, loss: 0.03026, acc: 0.98760, val_loss: 1.19646, val_accuracy: 0.82123


training loss: 0.02765, training accuracy: 0.98867: 100%|██████████| 668/668 [00:01<00:00, 480.57batch/s]


epoch 99, loss: 0.02765, acc: 0.98867, val_loss: 1.21770, val_accuracy: 0.82404


training loss: 0.02795, training accuracy: 0.98839: 100%|██████████| 668/668 [00:01<00:00, 452.33batch/s]


epoch 100, loss: 0.02795, acc: 0.98839, val_loss: 1.30863, val_accuracy: 0.81954
